In [1]:
import pandas as pd
import numpy as np

C:\Users\miche\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_cat = pd.read_excel('TRAIN_CATEGORICAL_METADATA_new.xlsx')
df_fun = pd.read_csv('TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
df_quant = pd.read_excel('TRAIN_QUANTITATIVE_METADATA_new.xlsx')
df_sol = pd.read_excel('TRAINING_SOLUTIONS.xlsx')

In [3]:
# need to missing values
print(df_cat.isnull().sum())
print(df_fun.isnull().sum())
print(df_quant.isnull().sum())
print(df_sol.isnull().sum())

participant_id                        0
Basic_Demos_Enroll_Year               0
Basic_Demos_Study_Site                0
PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
dtype: int64
participant_id          0
0throw_1thcolumn        0
0throw_2thcolumn        0
0throw_3thcolumn        0
0throw_4thcolumn        0
                       ..
196throw_198thcolumn    0
196throw_199thcolumn    0
197throw_198thcolumn    0
197throw_199thcolumn    0
198throw_199thcolumn    0
Length: 19901, dtype: int64
participant_id                  0
EHQ_EHQ_Total                  13
ColorVision_CV_Score           23
APQ_P_APQ_P_CP                 12
APQ_P_APQ_P_ID                 12
APQ_P_APQ_P_INV                12
APQ_P_APQ_P_OPD                12
APQ_P_APQ_P_PM                 12
A

In [4]:
# categorical dataset
# fill missing data with mode
categorical_cols = df_cat.select_dtypes(include=['float64']).columns
df_cat[categorical_cols] = df_cat[categorical_cols].fillna(df_cat[categorical_cols].mode().iloc[0])

# functional dataset
# fill missing data with 0
df_fun = df_fun.fillna(0)

# quantitative dataset
# fill missing data with median
numeric_cols = df_quant.select_dtypes(include=[np.number]).columns
df_quant[numeric_cols] = df_quant[numeric_cols].fillna(df_quant[numeric_cols].median())

# solutions dataset
df_sol = df_sol.fillna(df_sol.mode().iloc[0])

In [8]:
# do one-hot encoding for categorical columns/variables
df_cat.iloc[:, 1:] = df_cat.iloc[:, 1:].astype(str)

df_encoded = pd.get_dummies(df_cat.iloc[:, 1:], drop_first=True)
df_final = pd.concat([df_cat.iloc[:, [0]], df_encoded], axis=1)

     participant_id  Basic_Demos_Enroll_Year_2016  \
0      00aIpNTbG5uh                         False   
1      00fV0OyyoLfw                         False   
2      04X1eiS79T4B                         False   
3      05ocQutkURd6                         False   
4      06YUNBA9ZRLq                         False   
...             ...                           ...   
1208   zwjJWCRzKhDz                         False   
1209   zwXD5v17Rx01                         False   
1210   zWzLCi3NTBTd                         False   
1211   Zy9GTHDxUbXU                         False   
1212   Zye7yYRQohXi                         False   

      Basic_Demos_Enroll_Year_2017  Basic_Demos_Enroll_Year_2018  \
0                            False                         False   
1                             True                         False   
2                             True                         False   
3                            False                          True   
4                      

In [9]:
df_final.columns

Index(['participant_id', 'Basic_Demos_Enroll_Year_2016',
       'Basic_Demos_Enroll_Year_2017', 'Basic_Demos_Enroll_Year_2018',
       'Basic_Demos_Enroll_Year_2019', 'Basic_Demos_Enroll_Year_2020',
       'Basic_Demos_Study_Site_2', 'Basic_Demos_Study_Site_3',
       'Basic_Demos_Study_Site_4', 'PreInt_Demos_Fam_Child_Ethnicity_1.0',
       'PreInt_Demos_Fam_Child_Ethnicity_2.0',
       'PreInt_Demos_Fam_Child_Ethnicity_3.0',
       'PreInt_Demos_Fam_Child_Race_1.0', 'PreInt_Demos_Fam_Child_Race_10.0',
       'PreInt_Demos_Fam_Child_Race_11.0', 'PreInt_Demos_Fam_Child_Race_2.0',
       'PreInt_Demos_Fam_Child_Race_3.0', 'PreInt_Demos_Fam_Child_Race_4.0',
       'PreInt_Demos_Fam_Child_Race_7.0', 'PreInt_Demos_Fam_Child_Race_8.0',
       'PreInt_Demos_Fam_Child_Race_9.0', 'MRI_Track_Scan_Location_2.0',
       'MRI_Track_Scan_Location_3.0', 'MRI_Track_Scan_Location_4.0',
       'Barratt_Barratt_P1_Edu_15.0', 'Barratt_Barratt_P1_Edu_18.0',
       'Barratt_Barratt_P1_Edu_21.0', 'Barratt_B

In [11]:
merged_df = df_final.merge(df_fun, on="participant_id").merge(df_quant, on="participant_id").merge(df_sol, 
                                                                                                 on="participant_id")
merged_df

,participant_id,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Ethnicity_1.0,...,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan,ADHD_Outcome,Sex_F
0,00aIpNTbG5uh,False,False,False,True,False,False,False,True,True,...,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127,1,0
1,00fV0OyyoLfw,False,True,False,False,False,False,False,False,False,...,4.0,13.0,5.0,8.0,7.0,3.0,8.0,10.739219,1,0
2,04X1eiS79T4B,False,True,False,False,False,False,False,False,True,...,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381,0,1
3,05ocQutkURd6,False,False,True,False,False,False,False,False,False,...,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553,0,1
4,06YUNBA9ZRLq,False,False,True,False,False,False,False,False,False,...,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,False,False,False,True,False,False,False,True,True,...,5.0,13.0,5.0,8.0,7.0,2.0,8.0,11.272986,0,1
1209,zwXD5v17Rx01,False,False,True,False,False,False,False,False,False,...,0.0,5.0,0.0,3.0,0.0,0.0,7.0,7.364020,1,0
1210,zWzLCi3NTBTd,False,False,True,False,False,False,True,False,False,...,3.0,9.0,3.0,7.0,5.0,2.0,9.0,9.962126,1,1
1211,Zy9GTHDxUbXU,False,False,False,True,False,False,False,True,False,...,0.0,8.0,5.0,7.0,2.0,2.0,7.0,10.739219,1,0


In [14]:
merged_df_sub = merged_df.iloc[:, 1:]
merged_df_sub

,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Ethnicity_1.0,PreInt_Demos_Fam_Child_Ethnicity_2.0,...,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan,ADHD_Outcome,Sex_F
0,False,False,False,True,False,False,False,True,True,False,...,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127,1,0
1,False,True,False,False,False,False,False,False,False,False,...,4.0,13.0,5.0,8.0,7.0,3.0,8.0,10.739219,1,0
2,False,True,False,False,False,False,False,False,True,False,...,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381,0,1
3,False,False,True,False,False,False,False,False,False,False,...,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553,0,1
4,False,False,True,False,False,False,False,False,False,False,...,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,False,False,False,True,False,False,False,True,True,False,...,5.0,13.0,5.0,8.0,7.0,2.0,8.0,11.272986,0,1
1209,False,False,True,False,False,False,False,False,False,False,...,0.0,5.0,0.0,3.0,0.0,0.0,7.0,7.364020,1,0
1210,False,False,True,False,False,False,True,False,False,True,...,3.0,9.0,3.0,7.0,5.0,2.0,9.0,9.962126,1,1
1211,False,False,False,True,False,False,False,True,False,False,...,0.0,8.0,5.0,7.0,2.0,2.0,7.0,10.739219,1,0


In [17]:
merged_df_sub.columns

Index(['Basic_Demos_Enroll_Year_2016', 'Basic_Demos_Enroll_Year_2017',
       'Basic_Demos_Enroll_Year_2018', 'Basic_Demos_Enroll_Year_2019',
       'Basic_Demos_Enroll_Year_2020', 'Basic_Demos_Study_Site_2',
       'Basic_Demos_Study_Site_3', 'Basic_Demos_Study_Site_4',
       'PreInt_Demos_Fam_Child_Ethnicity_1.0',
       'PreInt_Demos_Fam_Child_Ethnicity_2.0',
       ...
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
       'MRI_Track_Age_at_Scan', 'ADHD_Outcome', 'Sex_F'],
      dtype='object', length=19973)

In [18]:
X = merged_df_sub.iloc[:, :-2]
y = merged_df_sub.iloc[:, -2:]

In [19]:
print(X.columns)
print(y.columns)

Index(['Basic_Demos_Enroll_Year_2016', 'Basic_Demos_Enroll_Year_2017',
       'Basic_Demos_Enroll_Year_2018', 'Basic_Demos_Enroll_Year_2019',
       'Basic_Demos_Enroll_Year_2020', 'Basic_Demos_Study_Site_2',
       'Basic_Demos_Study_Site_3', 'Basic_Demos_Study_Site_4',
       'PreInt_Demos_Fam_Child_Ethnicity_1.0',
       'PreInt_Demos_Fam_Child_Ethnicity_2.0',
       ...
       'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
       'MRI_Track_Age_at_Scan'],
      dtype='object', length=19971)
Index(['ADHD_Outcome', 'Sex_F'], dtype='object')


In [20]:
# need to cut down number of columns - most of the columns come from df_fun
# address collinearity? do PCA?
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
print(selector)
reduced_df = selector.fit_transform(X)
print(reduced_df.shape)

VarianceThreshold(threshold=0.01)
(1213, 19958)


In [21]:
corr_matrix = reduced_df.corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

df_reduced = reduced_df.drop(columns=to_drop)
print(df_reduced.shape)

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

In [22]:
reduced_df

array([[ 0.      ,  0.      ,  0.      , ...,  2.      ,  9.      ,
        14.274127],
       [ 0.      ,  1.      ,  0.      , ...,  3.      ,  8.      ,
        10.739219],
       [ 0.      ,  1.      ,  0.      , ...,  7.      ,  7.      ,
        13.463381],
       ...,
       [ 0.      ,  0.      ,  1.      , ...,  2.      ,  9.      ,
         9.962126],
       [ 0.      ,  0.      ,  0.      , ...,  2.      ,  7.      ,
        10.739219],
       [ 0.      ,  1.      ,  0.      , ...,  0.      ,  5.      ,
        10.739219]])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)  # Choose based on explained variance
reduced_features = pca.fit_transform(df)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X, y)
important_features = X.columns[np.argsort(model.feature_importances_)[-100:]]
df_reduced = df[important_features]
